# Práctica 2: Lectura y manipulación de datos con pandas

En esta práctica, se cargan los datos en marcos de datos (inglés: data frame) con la librería pandas:

[Pandas](https://pandas.pydata.org/)

para poder iniciar su procesamiento.

Como se mencionó en la [práctica 1](https://nbviewer.jupyter.org/github/alan-arnoldo-alcantar/DataScience_UANL/blob/master/p1/practica1.ipynb) el archivo`.csv` que contiene los datos crudos está organizado por columnas, donde cada columna corresponde con un factor del diseño de experimentos. El problema esta en que cada solucionario no utiliza todos los factores, por ello al entrenar con un solucionario en especifico puede que en algunas columnas (factores) aparezca NA, haciendo referencia que este factor no se utiliza. Por otra parte, la obtención de los datos se hizo en dos partes, por lo cual se tienen dos archivos`.scv` uno complementario del otro, que juntos tienen todos los resultados del diseño de experimentos.

En base a lo anterior las metas de esta práctica consisten en:
1. Cargar ambos archivos`.scv` utilizando la librería Pandas y unirlos en un solo marco de datos.
2. Generar un marco de datos para cada uno de los solucionarios que solo contengan los factores que utiliza el solucionario.
3. Una vez que se tengan los cuadros de datos por solucionario se deben reagrupar por factores, esto para que los resultados de los experimentos (precisión de la fase de entrenamiento, tiempo de entrenamiento y precisión en la fase de prueba) que tengas los mismos factores se agrupen en un solo renglón.
4. Guardar en un archivo`.scv` cada uno de los cuadros de datos que corresponden con un solucionario.

## Carga de los datos

In [376]:
import pandas as pd
pd.set_option('max_rows', 3)
datos1=pd.read_csv('../mlp_datos.csv')
datos2=pd.read_csv('../mlp_datos2.csv')
#ver que solucionarios aparecen en cada archivo
print(datos1.solver.unique())
print(datos2.solver.unique())
print(datos1)
print(datos2)

['sgd' 'adam' 'lbfgs']
['adam' 'lbfgs']
       label  partition solver  layer_num  neurona_num activation   alpha  \
0          1         10    sgd          1           30   logistic  0.0001   
...      ...        ...    ...        ...          ...        ...     ...   
23701  23702         20  lbfgs          7          210   logistic  1.0000   

       learning_rate_init learning_rate  power_t  train_accuracy  \
0                    0.01      constant      NaN        0.891333   
...                   ...           ...      ...             ...   
23701                 NaN           NaN      NaN        0.100000   

       trainig_time  test_accuracy  
0          2.679907         0.8812  
...             ...            ...  
23701      4.827969         0.0982  

[23702 rows x 13 columns]
        label  partition solver  layer_num  neurona_num activation  \
0       25153         20   adam          1           30   logistic   
...       ...        ...    ...        ...          ...        

## Union de cuadros de datos

Como se observa ambos cuadros de datos contienen las mismas columnas, de modo que para unir ambos cuadros de datos se utiliza la función `pd.concat()`, la cual une dos cuadros de datos que contengan las mismas columnas:

In [267]:
datos=pd.concat([datos1,datos2])
print(datos)

        label  partition solver  layer_num  neurona_num activation  \
0           1         10    sgd          1           30   logistic   
...       ...        ...    ...        ...          ...        ...   
29567  164160         90  lbfgs         10          210       tanh   

            alpha  learning_rate_init learning_rate  power_t  train_accuracy  \
0          0.0001                0.01      constant      NaN        0.891333   
...           ...                 ...           ...      ...             ...   
29567  10000.0000                 NaN           NaN      NaN        0.100000   

       trainig_time  test_accuracy  
0          2.679907         0.8812  
...             ...            ...  
29567     82.802846         0.0980  

[53270 rows x 13 columns]


In [17]:
#numero de renglones de datos1 + numero de renglones de datos2
23702+29568

53270

Vemos que efectivamente el número de renglones del cuadro de datos `datos` es igual a la suma de los renglones de los cuadros de datos `datos1` y `datos2`.

Ademas el titulo de la columna `trainig_time` debe ser `training_time`

In [268]:
datos=datos.rename(columns = {'trainig_time':'training_time'})
datos

,label,partition,solver,layer_num,neurona_num,activation,alpha,learning_rate_init,learning_rate,power_t,train_accuracy,training_time,test_accuracy
0,1,10,sgd,1,30,logistic,0.0001,0.01,constant,NaN,0.891333,2.679907,0.8812
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29567,164160,90,lbfgs,10,210,tanh,10000.0000,NaN,NaN,NaN,0.100000,82.802846,0.0980


## Dividiendo cuadro de datos
Lo que sigue es ver cuales son los solucionarios que se utilizaron y cuales factores usa cada uno.

In [269]:
datos.solver.unique()

array(['sgd', 'adam', 'lbfgs'], dtype=object)

In [270]:
datos.loc[datos.solver=='sgd']

,label,partition,solver,layer_num,neurona_num,activation,alpha,learning_rate_init,learning_rate,power_t,train_accuracy,training_time,test_accuracy
0,1,10,sgd,1,30,logistic,0.0001,0.01,constant,NaN,0.891333,2.679907,0.8812
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23327,23328,20,sgd,10,210,tanh,10000.0000,1.00,adaptive,NaN,0.100000,1503.814874,0.0980


In [271]:
datos.loc[datos.solver=='adam']

,label,partition,solver,layer_num,neurona_num,activation,alpha,learning_rate_init,learning_rate,power_t,train_accuracy,training_time,test_accuracy
1440,1441,10,adam,1,30,logistic,0.0001,0.01,NaN,NaN,0.767667,1.264617,0.7727
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29471,164064,90,adam,10,210,tanh,10000.0000,1.00,NaN,NaN,0.100000,82.034327,0.0982


In [272]:
datos.loc[datos.solver=='lbfgs']

,label,partition,solver,layer_num,neurona_num,activation,alpha,learning_rate_init,learning_rate,power_t,train_accuracy,training_time,test_accuracy
1728,1729,10,lbfgs,1,30,logistic,0.0001,NaN,NaN,NaN,0.927833,23.156492,0.8789
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29567,164160,90,lbfgs,10,210,tanh,10000.0000,NaN,NaN,NaN,0.100000,82.802846,0.0980


Aparentemente ninguno de los solucionarios utiliza el factor `power_t`, por lo que se analiza si existe algún renglón que tenga una entrada diferente a `NaN` en la columna `power_t`:

In [273]:
datos[~datos['power_t'].isin(['NaN'])]

,label,partition,solver,layer_num,neurona_num,activation,alpha,learning_rate_init,learning_rate,power_t,train_accuracy,training_time,test_accuracy
1,2,10,sgd,1,30,logistic,0.0001,0.01,invscaling,0.1,0.96200,8.691950,0.903
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23326,23327,20,sgd,10,210,tanh,10000.0000,1.00,invscaling,0.9,0.09725,1514.172772,0.104


Vemos que se debe tener mucho cuidado con el solucionario `sgd` porque aparentemente no utiliza el factor `power_t`, sin embargo como se ve, este factor se utiliza únicamente cuando el factor `learning_rate` tiene asignado el nivel `invscaling`.

A continuación, se pasa a crear los cuadros de datos por solucionario y de esta manera evitar columnas con entradas `NaN`

In [274]:
datos_adam=datos.loc[datos.solver=='adam',['partition','layer_num','neurona_num','activation','alpha',\
                                            'learning_rate_init','train_accuracy','training_time','test_accuracy']]
datos_adam

,partition,layer_num,neurona_num,activation,alpha,learning_rate_init,train_accuracy,training_time,test_accuracy
1440,10,1,30,logistic,0.0001,0.01,0.767667,1.264617,0.7727
...,...,...,...,...,...,...,...,...,...
29471,90,10,210,tanh,10000.0000,1.00,0.100000,82.034327,0.0982


In [377]:
datos_lbfgs=datos.loc[datos.solver=='lbfgs',['partition','layer_num','neurona_num','activation','alpha',\
                                            'train_accuracy','training_time','test_accuracy']]
datos_lbfgs

,partition,layer_num,neurona_num,activation,alpha,train_accuracy,training_time,test_accuracy
1728,10,1,30,logistic,0.0001,0.927833,23.156492,0.8789
...,...,...,...,...,...,...,...,...
29567,90,10,210,tanh,10000.0000,0.100000,82.802846,0.0980


In [276]:
datos_sgd_c=datos.loc[datos.learning_rate=='constant',['partition','layer_num','neurona_num','activation','alpha',\
                                                       'learning_rate_init','learning_rate','train_accuracy',\
                                                       'training_time','test_accuracy']]
datos_sgd_a=datos.loc[datos.learning_rate=='adaptive',['partition','layer_num','neurona_num','activation','alpha',\
                                                       'learning_rate_init','learning_rate','train_accuracy',\
                                                       'training_time','test_accuracy']]
datos_sgd_i=datos.loc[datos.learning_rate=='invscaling',['partition','layer_num','neurona_num','activation','alpha',\
                                                       'learning_rate_init','learning_rate','power_t','train_accuracy',\
                                                       'training_time','test_accuracy']]
datos_sgd_ca=pd.concat([datos_sgd_c,datos_sgd_a])
datos_sgd_i

,partition,layer_num,neurona_num,activation,alpha,learning_rate_init,learning_rate,power_t,train_accuracy,training_time,test_accuracy
1,10,1,30,logistic,0.0001,0.01,invscaling,0.1,0.96200,8.691950,0.903
...,...,...,...,...,...,...,...,...,...,...,...
23326,20,10,210,tanh,10000.0000,1.00,invscaling,0.9,0.09725,1514.172772,0.104


## Agrupando las variables de rendimiento

Durante el diseño de experimento se realizaron 10 repeticiones de cada combinación de factores para futuras pruebas de hipótesis, por lo que es necesario agrupar estos 10 resultados de rendimiento en un solo renglón que contenga la combinación de factores. Para ello se utiliza la función `groupby`.

In [396]:
pd.set_option('max_rows', 20)
datos_lbfgs_t=datos_lbfgs.groupby(['partition','layer_num','neurona_num','activation','alpha']).training_time.unique()
datos_lbfgs_t

partition  layer_num  neurona_num  activation  alpha     
10         1          30           logistic    0.0001        [23.15649223327637, 24.438762664794922, 20.000...
                                               1.0000        [21.88026475906372, 26.487568378448486, 20.667...
                                               10000.0000    [6.253906726837157, 8.277261734008789, 6.27513...
                                   tanh        0.0001        [22.267031908035282, 22.62187886238098, 17.399...
                                               1.0000        [22.207077026367188, 22.73449540138245, 17.455...
                                               10000.0000    [22.86952304840088, 23.37695956230164, 22.5856...
                      90           logistic    0.0001        [29.925067901611328, 32.30967140197754, 37.753...
                                               1.0000        [26.5259268283844, 30.80880570411682, 21.22847...
                                               10000.0

In [379]:
datos_lbfgs_t=datos_lbfgs_t.reset_index()
datos_lbfgs_t

,partition,layer_num,neurona_num,activation,alpha,training_time
0,10,1,30,logistic,0.0001,"[23.15649223327637, 24.438762664794922, 20.000..."
1,10,1,30,logistic,1.0000,"[21.88026475906372, 26.487568378448486, 20.667..."
2,10,1,30,logistic,10000.0000,"[6.253906726837157, 8.277261734008789, 6.27513..."
3,10,1,30,tanh,0.0001,"[22.267031908035282, 22.62187886238098, 17.399..."
4,10,1,30,tanh,1.0000,"[22.207077026367188, 22.73449540138245, 17.455..."
5,10,1,30,tanh,10000.0000,"[22.86952304840088, 23.37695956230164, 22.5856..."
6,10,1,90,logistic,0.0001,"[29.925067901611328, 32.30967140197754, 37.753..."
7,10,1,90,logistic,1.0000,"[26.5259268283844, 30.80880570411682, 21.22847..."
8,10,1,90,logistic,10000.0000,"[42.44521927833557, 16.72055459022522, 16.8898..."
9,10,1,90,tanh,0.0001,"[45.3847599029541, 34.61158514022827, 45.01342..."


Se hace lo mismo para `training_time` y `test_accuracy`

In [380]:
datos_lbfgs_ta=datos_lbfgs.groupby(['partition','layer_num','neurona_num','activation','alpha']).train_accuracy.unique()
datos_lbfgs_tt=datos_lbfgs.groupby(['partition','layer_num','neurona_num','activation','alpha']).test_accuracy.unique()
datos_lbfgs_tt=datos_lbfgs_tt.reset_index()
datos_lbfgs_ta=datos_lbfgs_ta.reset_index()

Una vez que se tienen agrupadas las variables de rendimiento por combinación de factores se unen en un solo cuadro de datos.

In [381]:
datos_lbfgs_t['train_accuracy']=datos_lbfgs_ta.train_accuracy
datos_lbfgs_t['test_accuracy']=datos_lbfgs_tt.test_accuracy
datos_lbfgs_t

,partition,layer_num,neurona_num,activation,alpha,training_time,train_accuracy,test_accuracy
0,10,1,30,logistic,0.0001,"[23.15649223327637, 24.438762664794922, 20.000...","[0.9278333333333332, 0.8973333333333333, 0.946...","[0.8789, 0.8536, 0.8941, 0.8863, 0.8487, 0.883..."
1,10,1,30,logistic,1.0000,"[21.88026475906372, 26.487568378448486, 20.667...","[0.9501666666666668, 0.9381666666666668, 0.939...","[0.8949, 0.8889, 0.8886, 0.8969, 0.8943, 0.89,..."
2,10,1,30,logistic,10000.0000,"[6.253906726837157, 8.277261734008789, 6.27513...","[0.802, 0.8371666666666666, 0.7961666666666667...","[0.8065, 0.826, 0.7957, 0.8123, 0.8238, 0.7471..."
3,10,1,30,tanh,0.0001,"[22.267031908035282, 22.62187886238098, 17.399...","[0.9263333333333332, 0.9081666666666668, 0.925...","[0.8854, 0.8706, 0.8778, 0.8876, 0.8799, 0.878..."
4,10,1,30,tanh,1.0000,"[22.207077026367188, 22.73449540138245, 17.455...","[0.91, 0.9313333333333332, 0.9356666666666666,...","[0.8725, 0.8844, 0.8905, 0.8834, 0.8837, 0.885..."
5,10,1,30,tanh,10000.0000,"[22.86952304840088, 23.37695956230164, 22.5856...","[0.9093333333333332, 0.9175, 0.926166666666666...","[0.8672, 0.8744, 0.878, 0.8705, 0.8532, 0.8589..."
6,10,1,90,logistic,0.0001,"[29.925067901611328, 32.30967140197754, 37.753...","[0.9915, 0.9898333333333332, 0.9945, 0.995, 0....","[0.9227, 0.9111, 0.92, 0.9163, 0.9149, 0.9136,..."
7,10,1,90,logistic,1.0000,"[26.5259268283844, 30.80880570411682, 21.22847...","[0.9918333333333332, 0.9925, 0.989833333333333...","[0.9287, 0.9269, 0.9274, 0.9258, 0.9251, 0.921..."
8,10,1,90,logistic,10000.0000,"[42.44521927833557, 16.72055459022522, 16.8898...","[0.8546666666666667, 0.868, 0.873, 0.877666666...","[0.8459, 0.8601, 0.8671, 0.8589, 0.8465, 0.864..."
9,10,1,90,tanh,0.0001,"[45.3847599029541, 34.61158514022827, 45.01342...","[0.9933333333333332, 0.987, 0.9888333333333332...","[0.9066, 0.9064, 0.9076, 0.9156, 0.9094, 0.911..."


## Guardar cuadro de datos
Para finalizar se guarda el cuadro de datos en un archivo`.csv` y se repite lo anterior para los cuadros de datos de los solucionarios `sgd` y `Adam`.

In [397]:
datos_lbfgs_t.to_csv("lbfgs.csv")